In [1]:
import pandas as pd
import numpy as np 
import json
from pandas.io.json import json_normalize
import os
from random import randint
import random



In [2]:
df_flat_grid = pd.read_csv('America_10km_inland.csv')

##### Open config.json as dataframe

In [3]:
with open('config_peril.json') as f:
    data = json.load(f)

In [4]:
#Open config.json as dataframe 
df_all = pd.DataFrame()
with open("config_peril.json") as file:
    json = json.load(file)
    
json






{'Required_fields': {'Constructioncode': [1000, 1001, 1002],
  'Occupancycode': [5000, 5001, 5100],
  'Peril': ['WTC']},
 'Countrycode': 'FR'}

#### Create list from json fields

In [5]:
required_fields = json['Required_fields']
countrycode = json['Countrycode']
peril = json['Required_fields']['Peril']
required_fields


{'Constructioncode': [1000, 1001, 1002],
 'Occupancycode': [5000, 5001, 5100],
 'Peril': ['WTC']}

In [6]:
peril



['WTC']

In [7]:
countrycode

'FR'

##### Rename 'FIPS' to 'CountryCode'

In [8]:
#Rename 'FIPS' to 'CountryCode'
df_flat_grid.rename(columns={'FIPS_10_': 'CountryCode'}, inplace=True)

In [9]:
df_flat_grid_country = df_flat_grid [df_flat_grid  ['CountryCode'] == countrycode]

In [10]:
df_flat_grid 

,FID_,FID_ne_10m,ADMIN,CountryCode,ISO_A2,ISO_A3,SUBREGION,FID_Americ,id,long,lat
0,NaN,11,Antarctica,AY,AQ,ATA,Antarctica,940,941,-71.95,-68.95
1,NaN,11,Antarctica,AY,AQ,ATA,Antarctica,941,942,-71.85,-68.95
2,NaN,11,Antarctica,AY,AQ,ATA,Antarctica,942,943,-71.75,-68.95
3,NaN,11,Antarctica,AY,AQ,ATA,Antarctica,943,944,-71.65,-68.95
4,NaN,11,Antarctica,AY,AQ,ATA,Antarctica,944,945,-71.55,-68.95
...,...,...,...,...,...,...,...,...,...,...,...
499075,NaN,92,Greenland,GL,GL,GRL,Northern America,1994995,1994996,-33.45,80.95
499076,NaN,92,Greenland,GL,GL,GRL,Northern America,1994996,1994997,-33.35,80.95
499077,NaN,92,Greenland,GL,GL,GRL,Northern America,1994997,1994998,-33.25,80.95
499078,NaN,92,Greenland,GL,GL,GRL,Northern America,1994998,1994999,-33.15,80.95


##### Reset index to place 'level_0' column

In [11]:

df_flat_grid_country = df_flat_grid_country.reset_index().reset_index()

##### Create OED Dataframe
##### Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'

In [12]:

df_oed = df_flat_grid_country[['level_0','lat','long','CountryCode']]

df_oed.columns = ['LocNumber','Latitude','Longitude','CountryCode']

#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed ['LocNumber']=df_oed ['LocNumber'] + 1
df_oed

/home/kamal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LocNumber,Latitude,Longitude,CountryCode
0,1,2.15,-54.15,FR
1,2,2.25,-54.45,FR
2,3,2.25,-54.35,FR
3,4,2.25,-54.25,FR
4,5,2.25,-54.15,FR
...,...,...,...,...
689,690,16.25,-61.45,FR
690,691,16.25,-61.35,FR
691,692,16.35,-61.75,FR
692,693,16.35,-61.45,FR


In [14]:
data

{'Required_fields': {'Constructioncode': [1000, 1001, 1002],
  'Occupancycode': [5000, 5001, 5100],
  'Peril': ['WTC']},
 'Countrycode': 'FR'}

In [15]:
df_oed

,LocNumber,Latitude,Longitude,CountryCode
0,1,2.15,-54.15,FR
1,2,2.25,-54.45,FR
2,3,2.25,-54.35,FR
3,4,2.25,-54.25,FR
4,5,2.25,-54.15,FR
...,...,...,...,...
689,690,16.25,-61.45,FR
690,691,16.25,-61.35,FR
691,692,16.35,-61.75,FR
692,693,16.35,-61.45,FR


In [16]:
portnum = 'FlatPF_{}'.format(countrycode)

In [17]:
df_oed = df_oed.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][0]),OccupancyCode=(data['Required_fields']["Occupancycode"][0]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")
##Create OED Dataframe

df_oed2 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed2.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed2 ['LocNumber']=df_oed2 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed2 = df_oed2.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][0]),OccupancyCode=(data['Required_fields']["Occupancycode"][1]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")


##Create OED Dataframe

df_oed3 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed3.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed3 ['LocNumber']=df_oed3 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed3 = df_oed3.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][0]),OccupancyCode=(data['Required_fields']["Occupancycode"][2]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")

##Create OED Dataframe

df_oed4 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed4.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed4 ['LocNumber']=df_oed4 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed4 = df_oed4.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][1]),OccupancyCode=(data['Required_fields']["Occupancycode"][0]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")

##Create OED Dataframe

df_oed5 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed5.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed5 ['LocNumber']=df_oed5 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed5 = df_oed5.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][1]),OccupancyCode=(data['Required_fields']["Occupancycode"][1]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")

df_oed6 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed6.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed6 ['LocNumber']=df_oed5 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed6 = df_oed6.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][1]),OccupancyCode=(data['Required_fields']["Occupancycode"][2]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")





df_oed7 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed7.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed7 ['LocNumber']=df_oed7 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed7 = df_oed7.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][2]),OccupancyCode=(data['Required_fields']["Occupancycode"][0]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")

df_oed8 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed8.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed8['LocNumber']=df_oed7 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed8 = df_oed8.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][2]),OccupancyCode=(data['Required_fields']["Occupancycode"][1]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")


df_oed9 = df_flat_grid_country[['level_0','lat','long','CountryCode']]

#Rename 'level_0' to 'LocNumber' & inherit columns from 'flat_grid'


df_oed9.columns = ['LocNumber','Latitude','Longitude','CountryCode']
#Drop 0 from index
#df_oed = df_oed.drop([0])
df_oed9 ['LocNumber']=df_oed9 ['LocNumber'] + 1

portnum = 'FlatPF_{}'.format(countrycode)

df_oed9 = df_oed9.assign(LocPerilsCovered=(data['Required_fields']["Peril"][0]),LocPeril=(data['Required_fields']["Peril"][0]),ConstructionCode=(data['Required_fields']["Constructioncode"][2]),OccupancyCode=(data['Required_fields']["Occupancycode"][2]),
                      LocCurrency='USD',BuildingTIV='100000',OtherTIV='1000',ContentsTIV='10000', CountryCode=countrycode,
                      BITIV='10000', PortNumber=portnum, AccNumber='1' , GeogScheme1="CRSL3", GeogName1="")

/home/kamal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/kamal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/kamal/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [18]:
frames = [df_oed, df_oed2, df_oed3, df_oed4, df_oed5,df_oed6,df_oed7,df_oed8,df_oed9]

In [19]:
result = pd.concat(frames)

In [20]:
sort = result.sort_values(['LocNumber'],ascending=True)

In [21]:
sort

,LocNumber,Latitude,Longitude,CountryCode,LocPerilsCovered,LocPeril,ConstructionCode,OccupancyCode,LocCurrency,BuildingTIV,OtherTIV,ContentsTIV,BITIV,PortNumber,AccNumber,GeogScheme1,GeogName1
0,1,2.15,-54.15,FR,WTC,WTC,1000,5000,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
0,1,2.15,-54.15,FR,WTC,WTC,1001,5001,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
0,1,2.15,-54.15,FR,WTC,WTC,1000,5100,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
0,1,2.15,-54.15,FR,WTC,WTC,1000,5001,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
0,1,2.15,-54.15,FR,WTC,WTC,1001,5000,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,694,16.35,-61.45,FR,WTC,WTC,1002,5001,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
692,694,16.35,-61.45,FR,WTC,WTC,1001,5100,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
693,694,16.45,-61.45,FR,WTC,WTC,1000,5001,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,
693,695,16.45,-61.45,FR,WTC,WTC,1002,5001,USD,100000,1000,10000,10000,FlatPF_FR,1,CRSL3,


In [23]:
sort.to_csv('FlatPort.csv', index=False)